In [2]:
!pip install steamspypi

In [4]:
import requests
import os
import json
import re
import time
import pandas as pd
import steamspypi
from sklearn.utils import shuffle

In [3]:
#Get all game ids and names
#URL call found here: https://partner.steamgames.com/doc/webapi/ISteamApps
URL = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'

response = requests.get(url=URL)
json_data = response.json()
GameIDs = pd.DataFrame.from_dict(json_data['applist']['apps'])
#Clean up the dataframe to remove empty strings and test/demo games
GameIDs['name'] = GameIDs['name'].str.strip()
GameIDs['name'] = GameIDs['name'].str.lower()
GameIDs = GameIDs[GameIDs['name'].isin(['','pieterw test app76 ( 216938 )','test2','test3', 'tidewoken public test', 
                                        'now testing: 407', 'test re(quietmansion1 special teaser)', '<h1>test</h1>', 
                                        'test', 'test project', 'steamvr performance test', 'testcontent', 'vrq test'
                                        ]) == False]
GameIDs = GameIDs[GameIDs['name'].str.contains('playtest')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('closed testing')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('testapp')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains(' test ')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('betatest')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('test server')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('beta test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('tidewoken public test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('open test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('dev test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('- test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('feature test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('technical test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('early access testing')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('_test')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains(' demo')==False]
GameIDs = GameIDs[GameIDs['name'].str.contains('public test')==False]

In [4]:
GameIDs1 = GameIDs.head(10)
GameIDs1.head(5)

,appid,name
9,1648210,apopia: prologue
10,1648220,m.a.r.s.s. soundtrack
11,1648230,inside the memories
13,1648250,freddy frog
14,1648310,pillars of protection


In [5]:
def CollectSteamData(GameIDDF, timeLimit_min):
    '''
    input: dataframe containing IDs and names of games 
    output: dataframe containing all api data from a random sample of the games
    '''
    #Steam API 1: primary game data
    #https://stackoverflow.com/questions/69512319/steam-api-to-get-game-info
    #Steam API 2: Review data
    #https://partner.steamgames.com/doc/store/getreviews
    #Steamspy API: Supplemental usage and cost data
    # https://pypi.org/project/steamspypi/
    # https://steamspy.com/api.php
    
    #Randomize the data frame
    IDs = shuffle(GameIDDF)
    GameDict = {}
    starttime = time.time()
    for appid in IDs['appid']:
        try:
            gameURL = 'http://store.steampowered.com/api/appdetails?appids=' + str(appid)
            response = requests.get(url=gameURL)
            json_data = response.json()
            GameData = json_data[str(appid)]['data']
            time.sleep(1) # 1 second rate limit on API calls
            
            data_request = dict()
            data_request['request'] = 'appdetails'
            data_request['appid'] = str(appid)
            steamspydata = steamspypi.download(data_request)

            # Combine all three json dictionaries and convert to dataframe
            
            GameData.update(steamspydata)
            time.sleep(1) # 1 second rate limit on API calls

        except: #games that do not have any associated data or other failed api calls
            time.sleep(1)
        endtime = time.time()
        elapsedtime = (endtime-starttime)/60
        if elapsedtime >= timeLimit_min: #If Greater than or equal to a set number of minutes, then end
            break
        #add all data for current app loop to GameDict
        GameDict.update({str(appid): GameData})
    #Convert to Dataframe
    GameDF = pd.DataFrame.from_dict(GameDict, orient='index')

    return GameDF

In [6]:
pd.options.display.max_columns = None 

In [7]:
Hours = 6
minutes = Hours*60
Sample_Game_Data1 = CollectSteamData(GameIDs1, minutes)

In [8]:
Sample_Game_Data1.head(10)

,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,release_date,support_info,background,content_descriptors,appid,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,languages,genre,tags,fullgame,reviews,legal_notice,achievements,controller_support
1647490,game,Mahjong Gold 2. Pirates Island,1647490,0,False,Captain on the bridge! Cast off the lines!<br ...,Captain on the bridge! Cast off the lines!<br ...,Let sail from Port Royal on a ship called the ...,"English, French, German, Russian",https://cdn.akamai.steamstatic.com/steam/apps/...,https://www.facebook.com/8FloorGames/,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],[Creobit],[8floor],"{'currency': 'USD', 'initial': 499, 'final': 4...",[585392],"[{'name': 'default', 'title': 'Buy Mahjong Gol...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256835838, 'name': 'Mahjong Gold 2. Pi...","{'coming_soon': False, 'date': 'May 27, 2021'}",{'url': 'https://www.facebook.com/8FloorGames'...,https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",1647490,Creobit,8floor,,1,0,0,"0 .. 20,000",0,0,0,0,499,499,0,1,"English, French, German, Russian",Casual,"{'Casual': 31, 'Singleplayer': 12, 'Adventure'...",NaN,NaN,NaN,NaN,NaN
1647470,dlc,Creature of Havoc (Fighting Fantasy Classics),1647470,0,False,<i>Evil is festering in Trolltooth Pass. The n...,<i>Evil is festering in Trolltooth Pass. The n...,The necromancer Zharradan Marr is close to ste...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://bit.ly/TMGdevblog,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[Tin Man Games],[Tin Man Games],"{'currency': 'USD', 'initial': 399, 'final': 3...",[585386],"[{'name': 'default', 'title': 'Buy Creature of...","{'windows': True, 'mac': True, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '23', 'description': 'Indie'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,"{'coming_soon': False, 'date': 'Aug 26, 2021'}","{'url': '', 'email': 'minion@tinmangames.com.au'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",1647470,Tin Man Games,Tin Man Games,,0,0,0,"0 .. 20,000",0,0,0,0,399,399,0,0,English,"Indie, RPG",[],"{'appid': '856880', 'name': 'Fighting Fantasy ...",“This book has what must be one of the densest...,"© Tin Man Games Pty Ltd, 2021<br />\r\nFightin...",NaN,NaN
1647430,game,Black Mansion,1647430,0,False,Get ready to solve the puzzles walking through...,Get ready to solve the puzzles walking through...,The shadow of death hidden in the mansion... S...,"English, Japanese, Korean, Simplified Chinese",https://cdn.akamai.steamstatic.com/steam/apps/...,None,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],[PICTOLOGI],[PICTOLOGI],"{'currency': 'USD', 'initial': 699, 'final': 6...",[585373],"[{'name': 'default', 'title': 'Buy Black Mansi...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256836153, 'name': 'Play', 'thumbnail'...","{'coming_soon': False, 'date': 'Jun 15, 2021'}","{'url': '', 'email': 'pictologisteam@gmail.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'The black mansion in...",1647430,PICTOLOGI,PICTOLOGI,,1,0,0,"0 .. 20,000",0,0,0,0,699,699,0,0,"English, Japanese, Korean, Simplified Chinese","Action, Adv

In [9]:
Sample_Game_Data1.shape

(10, 52)

In [10]:
from google.colab import files
Sample_Game_Data1.to_csv('RawGameData1.csv') 
files.download('RawGameData1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
Hours = 6
minutes = Hours*60
Sample_Game_Data = CollectSteamData(GameIDs, minutes)

In [12]:
Sample_Game_Data.head(10)

,type,name,steam_appid,required_age,is_free,controller_support,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,publishers,package_groups,platforms,categories,release_date,support_info,background,content_descriptors,appid,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,languages,genre,tags,legal_notice,developers,genres,screenshots,movies,price_overview,packages,achievements,dlc,reviews,recommendations,ext_user_account_notice,metacritic,demos,drm_notice,alternate_appid
832484,episode,Blender Character Rigging for Beginners HD: Bu...,832484,0,False,full,,,Build Foot Roll Rig - Part 1,"{'appid': '829200', 'name': 'Blender Character...",English<strong>*</strong><br><strong>*</strong...,https://cdn.akamai.steamstatic.com/steam/apps/...,None,[],[],[],[],[],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 28, 'description': 'Full controller su...","{'coming_soon': False, 'date': 'Mar 27, 2018'}","{'url': '', 'email': ''}",,"{'ids': [], 'notes': None}",832484,,,,3,0,0,"0 .. 20,000",0,0,0,0,0,0,0,0,English,,"{'Design & Illustration': 26, 'Animation & Mod...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1708320,game,GRAPPIN,1708320,0,False,full,"<img src=""https://cdn.akamai.steamstatic.com/s...","<img src=""https://cdn.akamai.steamstatic.com/s...","After a difficult awakening, you stumble upon ...",NaN,"English, French, Japanese",https://cdn.akamai.steamstatic.com/steam/apps/...,https://twitter.com/grappingame,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],[Ahmin Hafidi],[],"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","{'coming_soon': True, 'date': '2022'}","{'url': 'https://grappingame.tumblr.com/', 'em...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",1708320,,,,0,0,0,"0 .. 20,000",0,0,0,0,None,None,None,0,None,,[],GRAPPIN uses Unreal® Engine. Unreal® is a trad...,"[Ahmin Hafidi, Benoit Malis]","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256844414, 'name': 'GRAPPIN - Announce...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1556670,game,I'm a King,1556670,0,False,NaN,"A young prince, whose father suddenly fell ill...","A young prince, whose father suddenly fell ill...",Role-playing strategy &quot;I'm a King&quot;. ...,NaN,"English, Russian",https://cdn.akamai.steamstatic.com/steam/apps/...,None,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],[Alexandr Getmanskiy],"[{'name': 'default', 'title': 'Buy I'm a King'...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","{'coming_soon': False, 'date': 'Apr 22, 2021'}","{'url': '', 'email': 'imaking.dev@gmail.com'}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",1556670,Alexandr Getmanskiy,Alexandr Getmanskiy,,3,6,0,"0 .. 20,000",0,0,0,0,999,999,0,0,"English, Russian","Adventure, Indie, RPG, Simulation, Strategy","{'Choose Your Own Adventure': 193, 'Political ...",NaN,[Alexandr Getmanskiy],"[{'id': '25', 'description': 'Adventure'}, {'i...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 256830352, 'name': 'Main trailer', 'th...","{'currency': 'USD', 'initial': 999, 'final': 9...",[549794],"{'total': 14, 'highlighted': [{'name': 'KING N...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1171120,game,Nira,1171120,0,False,full,"<img src=""https://cdn.akamai.steamstatic.com/s...","<img src=""https://cdn.akamai.steamstatic.com/s...",Explore procedurally generated islands in this...,NaN,"English, French, German, Spanish - Spain, Port...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://baselinegames.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],[Graffiti Games],"[{'name': 'default', 'title'

In [13]:
from google.colab import files
Sample_Game_Data.to_csv('RawGameData_full.csv') 
files.download('RawGameData_full.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
Sample_Game_Data.columns

Index(['type', 'name', 'steam_appid', 'required_age', 'is_free',
       'controller_support', 'detailed_description', 'about_the_game',
       'short_description', 'fullgame', 'supported_languages', 'header_image',
       'website', 'pc_requirements', 'mac_requirements', 'linux_requirements',
       'publishers', 'package_groups', 'platforms', 'categories',
       'release_date', 'support_info', 'background', 'content_descriptors',
       'appid', 'developer', 'publisher', 'score_rank', 'positive', 'negative',
       'userscore', 'owners', 'average_forever', 'average_2weeks',
       'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
       'ccu', 'languages', 'genre', 'tags', 'legal_notice', 'developers',
       'genres', 'screenshots', 'movies', 'price_overview', 'packages',
       'achievements', 'dlc', 'reviews', 'recommendations',
       'ext_user_account_notice', 'metacritic', 'demos', 'drm_notice',
       'alternate_appid'],
      dtype='object')